In [2]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
import random

2023-09-07 13:16:11.270989: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-07 13:16:11.372053: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-07 13:16:11.373210: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 13:16:13.035541: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

In [4]:


train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats') 
train_cats_dir = os.path.join(train_dir, 'cats') 
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

#print('total training cat images:', num_cats_tr)
#print('total training dog images:', num_dogs_tr)

#print('total validation cat images:', num_cats_val)
#print('total validation dog images:', num_dogs_val)
#print("--")
#print("Total training images:", total_train)
#print("Total validation images:", total_val)



In [8]:
epochs = 1
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [6]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [7]:
train_data_gen = train_image_generator.flow_from_directory(directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [14]:
class Individual:
    def __init__(self):
        self.numLayers = random.randrange(2,5)
        self.numNodes = random.randrange(8,32)
        self.model = self.create_model()
        self.acc = self.calc_acc()

    def create_model(self):
        modelList = [Conv2D(self.numNodes, 3, padding='same', activation='relu', input_shape=(150, 150, 3)),
                     MaxPooling2D()]
        
        for _ in range(self.numLayers):
            modelList.append(Conv2D(self.numNodes, 3, padding='same', activation='relu'))
            modelList.append(MaxPooling2D())
        
        modelList.append(Flatten())
        modelList.append(Dense(512, activation='relu'))
        modelList.append(Dense(1, activation='sigmoid'))
        
        model = Sequential(modelList)
        model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
        
        #model.summary()
        
        return model
        
    def calc_acc(self):
        print(self.numLayers, self.numNodes)
        history = self.model.fit(train_data_gen,
                    steps_per_epoch=total_train // batch_size,
                    epochs=epochs,
                    validation_data=val_data_gen,
                    validation_steps=total_val // batch_size)
        return history.history['accuracy'][-1]
    
    def __lt__(self,other):
        return self.acc < other.acc

In [15]:
def selection(population, tour_size):
    competitors_idx = random.sample(range(len(population)), tour_size)
    #print(competitors_idx)
    
    max_acc = -1.0
    max_idx = -1
    for i in range(tour_size):
        if population[competitors_idx[i]].acc > max_acc:
            max_acc = population[competitors_idx[i]].acc
            max_idx = competitors_idx[i]
    return max_idx

In [16]:
def crossover(individual_1, individual_2, child_1, child_2):
    child_1.numLayers = individual_1.numLayers
    child_1.numNodes = individual_2.numNodes
    
    child_2.numLayers = individual_2.numLayers
    child_2.numNodes = individual_1.numNodes
    

In [17]:
def mutation():
    pass

In [18]:
pop_size = 5
num_iters = 1
elitism = 1
mutation_prob = 0.05
tour_size = 2

population = [Individual() for _ in range(pop_size)]
new_population = population[:]

print([(population[i].numLayers, population[i].numNodes) for i in range(pop_size)])
    
for _ in range(num_iters):
    
    population.sort()
    new_population[:elitism] = population[:elitism]
    
    for i in range(elitism,pop_size,2):

        parent1_idx = selection(population, tour_size)
        parent2_idx = selection(population, tour_size)
        
        #print(parent1_idx, parent2_idx)
        
        crossover(population[parent1_idx], population[parent2_idx], new_population[i], new_population[i+1])
        
        #mutation(population[i], mutation_prob)
        #mutation(population[i+1], mutation_prob)
        
        new_population[i].model = new_population[i].create_model()
        new_population[i+1].model = new_population[i+1].create_model()
        
        #print(f"new_pop{i}:{new_population[i].numLayers},{new_population[i].numNodes}")
        #new_population[i].model.summary()
        #print(f"new_pop{i+1}:{new_population[i+1].numLayers},{new_population[i+1].numNodes}")
        #new_population[i+1].model.summary()
        
        new_population[i].acc = new_population[i].calc_acc()
        new_population[i+1].acc = new_population[i+1].calc_acc()
    
    population[:] = new_population[:]
    print([(population[i].numLayers, population[i].numNodes) for i in range(pop_size)])

best_individual = max(population)
print(best_individual.numLayers, best_individual.numNodes, best_individual.acc)

4 20


/tmp/ipykernel_41987/330682070.py:29: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = self.model.fit_generator(train_data_gen,


15/15 [==============================] - 14s 751ms/step - loss: 0.7038 - accuracy: 0.5129 - val_loss: 0.6945 - val_accuracy: 0.5268
2 9
15/15 [==============================] - 9s 492ms/step - loss: 0.6972 - accuracy: 0.5250 - val_loss: 0.6869 - val_accuracy: 0.5982
4 31
15/15 [==============================] - 17s 997ms/step - loss: 0.6949 - accuracy: 0.4833 - val_loss: 0.6915 - val_accuracy: 0.4955
4 26
15/15 [==============================] - 15s 867ms/step - loss: 0.6957 - accuracy: 0.5237 - val_loss: 0.6941 - val_accuracy: 0.4777
2 10
15/15 [==============================] - 9s 472ms/step - loss: 0.7111 - accuracy: 0.5104 - val_loss: 0.6858 - val_accuracy: 0.5491
[(4, 20), (2, 9), (4, 31), (4, 26), (2, 10)]
4 20
15/15 [==============================] - 13s 766ms/step - loss: 0.6975 - accuracy: 0.5146 - val_loss: 0.6935 - val_accuracy: 0.5223
4 26
15/15 [==============================] - 16s 978ms/step - loss: 0.7012 - accuracy: 0.4771 - val_loss: 0.6926 - val_accuracy: 0.5446
4 26